<a href="https://colab.research.google.com/github/EminedEm/EminedEm/blob/main/demirciemine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

import torch
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install --no-deps packaging ninja einops "flash-attn>=2.6.3"

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True

fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-2-9b",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.9.post4: Fast Gemma2 patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/6.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.9.post4 patched 42 layers with 42 QKV layers, 42 O layers and 42 MLP layers.


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving Bluearf Data Science Coding Task Data.xlsx to Bluearf Data Science Coding Task Data.xlsx


In [ ]:
import pandas as pd

file_path = 'Bluearf Data Science Coding Task Data.xlsx'
data = pd.read_excel(file_path)
print("Data loaded successfully!")


Data loaded successfully!


In [ ]:
data_cleaned = data[['POTENTIAL_USE_CASES', 'ACTIVITY']].dropna()

data_cleaned['POTENTIAL_USE_CASES'] = data_cleaned['POTENTIAL_USE_CASES'].str.lower().str.strip()
data_cleaned['ACTIVITY'] = data_cleaned['ACTIVITY'].str.lower().str.strip()

print(data_cleaned.head())


                                 POTENTIAL_USE_CASES  \
0  doğal su yönetimi, su arıtma, su temini, doğal...   
1  perakende ticaret, mağaza yönetimi, perakende ...   
2  toptan ticaret, perakende satış, ürün toptancı...   
3  atık yönetimi, çöp bertarafı, geri dönüşüm hiz...   
4  kanalizasyon hizmetleri, atık su yönetimi, kan...   

                                            ACTIVITY  
0  natural water/water treatment and supply services  
1  wholesale and retail trade and repair services...  
2  wholesale trade services/except of motor vehic...  
3  remediation services and other waste managemen...  
4                    sewerage services/sewage sludge  


In [ ]:
csv_file_path = 'Bluearf_Data_Cleaned.csv'
data_cleaned.to_csv(csv_file_path, index=False)

from google.colab import files
files.download(csv_file_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install datasets
from datasets import Dataset

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    outputs = examples["output"]
    texts = []
    for instruction, output in zip(instructions, outputs):
        text = alpaca_prompt.format(instruction, "", output) + EOS_TOKEN
        texts.append(text)
    return {"text": texts}

data_dict = {"instruction": data_cleaned['POTENTIAL_USE_CASES'].tolist(),
             "output": data_cleaned['ACTIVITY'].tolist()}

dataset = Dataset.from_dict(data_dict)

formatted_dataset = dataset.map(formatting_prompts_func, batched=True)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

tokenized_dataset = formatted_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/255 [00:00<?, ? examples/s]

Map:   0%|          | 0/255 [00:00<?, ? examples/s]

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

KeyError: 'text'

In [ ]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=tokenized_dataset,
    dataset_text_field="text",
    max_seq_length=512,
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=10,
        num_train_epochs=3,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=10,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
    ),
)


In [ ]:
trainer.train()


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 255 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 4
\        /    Total batch size = 4 | Total steps = 189
 "-____-"     Number of trainable parameters = 54,018,048


Step,Training Loss
10,2.622400
20,0.382000
30,0.257300
40,0.283800
50,0.159100
60,0.155400
70,0.164800
80,0.118700
90,0.112500
100,0.106900


TrainOutput(global_step=189, training_loss=0.27107994739340724, metrics={'train_runtime': 1578.2005, 'train_samples_per_second': 0.485, 'train_steps_per_second': 0.12, 'total_flos': 2.1588686955085824e+16, 'train_loss': 0.27107994739340724, 'epoch': 2.9647058823529413})

In [ ]:
custom_prompt = """Below is an instruction describing a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

example_use_case = "atık yönetimi, çöp bertarafı, geri dönüşüm hizmetleri"  # Example from your dataset

FastLanguageModel.for_inference(model)  # Enable fast inference

inputs = tokenizer(
    [
        custom_prompt.format(
            example_use_case,
            "",
            ""
        )
    ], return_tensors="pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)

predicted_activity = tokenizer.batch_decode(outputs, skip_special_tokens=True)

print("Predicted Activity:", predicted_activity[0])


Predicted Activity: Below is an instruction describing a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
atık yönetimi, çöp bertarafı, geri dönüşüm hizmetleri

### Input:


### Response:
waste collection/treatment and disposal services/materials recovery services


In [ ]:
from google.colab import files
import shutil

shutil.make_archive('lora_model', 'zip', 'lora_model')

files.download('lora_model.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model",
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model)


In [ ]:

example_use_case = "atık yönetimi, çöp bertarafı, geri dönüşüm hizmetleri"  # Example from your dataset

inputs = tokenizer(
    [
        custom_prompt.format(
            example_use_case,  )
            "",
            ""
        )
    ], return_tensors="pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)

predicted_activity = tokenizer.batch_decode(outputs, skip_special_tokens=True)

print("Predicted Activity:", predicted_activity[0])



Predicted Activity: Below is an instruction describing a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
atık yönetimi, çöp bertarafı, geri dönüşüm hizmetleri

### Input:


### Response:
waste collection/treatment and disposal services/materials recovery services


In [ ]:
outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)


In [ ]:
tokenizer.batch_decode(outputs)

['<bos>Below is an instruction describing a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\natık yönetimi, çöp bertarafı, geri dönüşüm hizmetleri\n\n### Input:\n\n\n### Response:\nwaste collection/treatment and disposal services/materials recovery services<eos>']

In [ ]:
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder

true_activities = [...]
predicted_activities = [...]

label_encoder = LabelEncoder()

all_labels = list(set(true_activities + predicted_activities))
label_encoder.fit(all_labels)

true_activities_encoded = label_encoder.transform(true_activities)
predicted_activities_encoded = label_encoder.transform(predicted_activities)

print("Accuracy:", accuracy_score(true_activities_encoded, predicted_activities_encoded))
print(classification_report(true_activities_encoded, predicted_activities_encoded))

Accuracy: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1

    accuracy                           1.00         1
   macro avg       1.00      1.00      1.00         1
weighted avg       1.00      1.00      1.00         1



In [ ]:
!pip install flask-ngrok
